In [1]:
!pip install accelerate transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00


In [38]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

path_to_facts = "/content/drive/MyDrive/NLP_FINAL_PROJECT/OpenBookQA-V1-Sep2018/Data/Additional/crowdsourced-facts.txt"

with open(path_to_facts, 'r') as f:
    text = f.read()

facts = text.splitlines()[1:] # drop the empty line at the top of the file

facts[:10]

Mounted at /content/drive/


['-2+2=0',
 '1 day has 1440 minutes',
 '1 out of 4 people in the world are chronically hungry.',
 '100 degrees celsius is the boiling point of water',
 '12:00 PM is noon',
 '168 hours is a week',
 '2008 to 2018 is a decade',
 '24 is most of 30',
 '3 meals a day is standard',
 "35 percent of the world's food crops depend on animal pollinators to reproduce"]

In [39]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.5 MB/s eta 0:00:00


In [58]:
# https://arxiv.org/abs/1908.10084
# https://www.sbert.net/
# https://huggingface.co/sentence-transformers

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("all-MiniLM-L6-v2")

In [60]:
# Our sentences to encode
facts

# Sentences are encoded by calling model.encode()
fact_embeddings = model.encode(facts, convert_to_tensor=True)

In [48]:
from datasets import load_dataset

dataset = load_dataset("allenai/openbookqa", name="additional")

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question_stem', 'choices', 'answerKey', 'fact1', 'humanScore', 'clarity', 'turkIdAnonymized'],
        num_rows: 4957
    })
    validation: Dataset({
        features: ['id', 'question_stem', 'choices', 'answerKey', 'fact1', 'humanScore', 'clarity', 'turkIdAnonymized'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'question_stem', 'choices', 'answerKey', 'fact1', 'humanScore', 'clarity', 'turkIdAnonymized'],
        num_rows: 500
    })
})

In [55]:
dataset["train"][:2]

{'id': ['7-980', '7-584'],
 'question_stem': ['The sun is responsible for',
  'When standing miles away from Mount Rushmore'],
 'choices': [{'text': ['puppies learning new tricks',
    'children growing up and getting old',
    'flowers wilting in a vase',
    'plants sprouting, blooming and wilting'],
   'label': ['A', 'B', 'C', 'D']},
  {'text': ['the mountains seem very close',
    'the mountains are boring',
    'the mountains look the same as from up close',
    'the mountains seem smaller than in photographs'],
   'label': ['A', 'B', 'C', 'D']}],
 'answerKey': ['D', 'D'],
 'fact1': ['the sun is the source of energy for physical cycles on Earth',
  'as distance to an object increases , that object will appear smaller'],
 'humanScore': [1.0, 1.0],
 'clarity': [2.0, 1.399999976158142],
 'turkIdAnonymized': ['b356d338b7', 'b356d338b7']}

In [64]:
import torch

In [84]:
# https://www.sbert.net/docs/usage/semantic_textual_similarity.html

batch = dataset["train"][:5]
print(batch["question_stem"])
print(batch["fact1"])
stem_embeddings = model.encode(batch["question_stem"], convert_to_tensor=True)

# compute cosine similarities
cosine_scores = util.cos_sim(stem_embeddings, fact_embeddings)

# output the pairs with their score
for i in range(len(batch["question_stem"])):
  best = torch.topk(cosine_scores, k=3, dim=-1).indices[i]
  print("{} \t {} \t {} \t {}".format(
      batch["question_stem"][i], facts[best[0]], facts[best[1]], facts[best[2]]
  ))

['The sun is responsible for', 'When standing miles away from Mount Rushmore', 'When food is reduced in the stomach', 'Stars are', 'You can make a telescope with a']
['the sun is the source of energy for physical cycles on Earth', 'as distance to an object increases , that object will appear smaller', 'digestion is when stomach acid breaks down food', 'a star is made of gases', 'Galileo Galilei made improvements to the telescope to make better observations of celestial bodies']
The sun is responsible for 	 the sun is in the sky 	 the sun is a producer of heat 	 the sun is a source of heat
When standing miles away from Mount Rushmore 	 a mile away is quite far 	 you can see further on top of a mountain 	 4 feet is the closest distance of the 4 options
When food is reduced in the stomach 	 eaten food is turned into a liquid by the stomach 	 stomach acid breaks down food substances 	 food needs to be digested to be passed out properly
Stars are 	 stars are a part of the sky 	 celestial bo